# Predicting Steam Game Review Scores  

The goal of this project is to analyze **whether structured attributes carry predictive power when predicting game reviews.**  

Some of the key questions I aim to answer are:  

- 🤖 **Predictive Power:** How effectively can structured attributes predict game reviews?
- 💡 **Interpretability:** What are the most influential features? What can they tell us about player satisfaction?  
- 🚫 **Insight Through Failure:** What are the least influential features? What do weak correlations mean?
- 🔍 **Human Predictability:** What does this suggest about how predictable, or unpredictable, human sentiment data truly is?    
  
---

### 📊 Modeling & Methodology  

1️⃣ **Target Definition:** Categorize review scores into classes: *High*, *Medium*, and *Low*, based on rating distributions.

2️⃣ **Feature Engineering:** Transform Steam data into model ready features: log-scaled review counts, log-scaled release prices, and release (year, month, season).  

3️⃣ **Model Development:** Train and evaluate a Random Forest to test predictability.  

4️⃣ **Evaluation & Insights:** Use model performance to determine if structured attributes meaningfully predict review outcomes, and identify which features carry the most predictive weight.  

## Section 1: Data Preprocessing

In [1]:
# Start by importing necessary libraries. 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display
# scikit-learn modules
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, f1_score, classification_report, confusion_matrix, balanced_accuracy_score)

KeyboardInterrupt: 

To get started, we'll import the dataset for this analysis. It can be found inside the .zip folder of this notebook, or, on kaggle.com at: https://www.kaggle.com/datasets/benjaminlundkvist/steam-sales-historical-dataset. 

- **DISCLAIMER**: Dataset updates weekly on Kaggle. This notebook is built using the dataset that was updated on September 29th, 2025.

In [ ]:
# Load & prepare data.
steam_sales = pd.read_csv('steam_sales_Sep28.csv')

# Column names & row quantity. 
print("Columns:", list(steam_sales.columns))
print("Rows:", steam_sales.shape[0])

First, we'll check for null and duplicate values, as well as the data types.

In [ ]:
# Check for null and duplicate values. 
print("Nulls:", steam_sales.isnull().values.any())
print("Duplicates:", steam_sales[steam_sales['Game Name'].duplicated(keep=False)].values.any())

Data is pulled directly from the Steam API and refreshed weekly, so the absence of null values and the presence of duplicates are both expected.

In [ ]:
# Check data types.
print(steam_sales.dtypes.to_string())

Reviews, Release Date, and Fetched At are stored as objects (strings). #Reviews will be converted to numeric, as it represents quantitative data. 

Release Date and Fetched At will be converted to datetime to take advantage of pandas’ time and date functionality.

In [ ]:
# Convert Reviews to numeric. 
steam_sales['#Reviews'] = pd.to_numeric(steam_sales['#Reviews'].str.replace(',', ''))

# Convert Fetched At to datetime.
steam_sales['Fetched At'] = pd.to_datetime(steam_sales['Fetched At'])

# Convert Release Date to datetime.  
date_format_1 = pd.to_datetime(steam_sales['Release Date'], errors='coerce', format = '%b %d, %Y').dropna() # Month Day, Year.
date_format_2 = pd.to_datetime(steam_sales['Release Date'], errors='coerce', format = '%d %b, %Y').dropna() # Day Month, Year. (Only used by 1st refresh)
steam_sales['Release Date'] = pd.concat([date_format_2, date_format_1])

Duplicate values are likely a consequence of the dataset's weekly updates, where new data is appended, rather than resetting each week. 

Not all games are tracked each week however, some games may have no duplicates while others might have multiple.

In [ ]:
# Filter duplicates.
steam_sales = steam_sales.sort_values('Fetched At', ascending = False)
steam_sales = steam_sales.drop_duplicates(subset=['Game Name'], keep='first')
steam_sales = steam_sales.reset_index(drop=True)

# Check for nulls and duplicates once more.
print("Nulls:", steam_sales.isnull().values.any())
print("Duplicates:", steam_sales[steam_sales['Game Name'].duplicated(keep=False)].values.any())

Duplicate values gone! We did this using Fetched At to sort recent updates to the top, then, keeping only the first iteration, dropping values with identical names.

Before moving on, let's preview the changes and call `.describe()` to get a sense of what we're about to work with.

In [ ]:
# Preview and description of cleaned dataset. 
display(steam_sales.head(3))
display(steam_sales.describe().round(2))

# Section 2: Creating Analysis Variables

Now that our data is clean, we can define the variables we'll be modeling on. These will go into a new dataframe exclusively for analysis variables. 


In [ ]:
# Creating analysis variables dataframe.
analysis_df = pd.DataFrame()

The data description suggests a potential skew, so we'll start by examining the Original Price (€) and #Reviews features to check whether they require transformation.

In [ ]:
# Histogram for Original Price.
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(steam_sales['Original Price (€)'], bins=50, color='salmon', edgecolor='black')
ax.set_xlabel('Original Price (€)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Original Price')
plt.savefig('output/hist_original_price.png', dpi=150, bbox_inches='tight')
plt.show()

Game prices cluster towards €20, with the highest at €100. The distribution isn't severely skewed, but it could hurt the model's results.

In [ ]:
# Histogram for #Reviews.
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(steam_sales['#Reviews'], bins=50, color='skyblue', edgecolor='black')
ax.set_xlabel('Number of Reviews (in millions)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Reviews')
plt.savefig('output/hist_reviews.png', dpi=150, bbox_inches='tight')
plt.show()

Reviews, however, has a severe skew. With a max value at over a million and most games not even approaching 100,000, this needs to be transformed. 

To ensure values aren't ignored while others get overvalued, both features will be using log-transformed data. 

In [ ]:
# Log-transformed #Reviews.
analysis_df['Log Reviews'] = np.log1p(steam_sales['#Reviews'])

# Log-transformed Original Price.
analysis_df['Log Price'] = np.log1p(steam_sales['Original Price (€)'])

From Release Date, we'll be using Year for annual trends, and Month to analyze the impact of release timing.

We'll also use Discount% and operating system support, flipping Discounts from negative to positive, and adding OS Support as is.

In [ ]:
# Year & Month as features.
analysis_df['Year'] = steam_sales['Release Date'].dt.year
analysis_df['Month'] = steam_sales['Release Date'].dt.month

# Positive Discount% values.
analysis_df['Discount'] = steam_sales['Discount%'].abs()

# Add OS Support columns.
analysis_df['Windows'] = steam_sales['Windows']
analysis_df['Linux'] = steam_sales['Linux']
analysis_df['MacOS'] = steam_sales['MacOS']
analysis_df['OS_Count'] = analysis_df[['Windows','Linux','MacOS']].sum(axis=1)

To use ratings as a target variable, the scale needs clarification. Steam displays reviews as 9 different text categories, while the data represents them numerically, from 3.0 to 8.0.

By cross-referencing several titles with their Steam store pages, the following mapping emerges:

- 3.0: *Mostly Negative*
- 4.0: *Mixed*
- 5.0: *Mostly Positive*
- 6.0: *Positive*
- 7.0: *Very Positive*
- 8.0: *Overwhelmingly Positive*

Extremely low ratings, (below 3.0) are absent in the data. Likely the result of a filter, games *this* bad often carry extreme quality issues or customer deception.

For our target variable, scores are grouped into three classes:
- *Low* (3.0–4.0), *Medium* (5.0–6.0), and *High* (7.0–8.0).

In [ ]:
# Create target classes: *Low*, *Medium*, *High*. 
analysis_df['Review_Category'] = pd.cut(steam_sales['Rating'], bins=[2.5, 4.5, 6.5, 8.5], labels=['Low', 'Medium', 'High'])

# Verify analysis variables dataframe. 
analysis_df.head()

Dataframe looks ready! Before continuing, we'll verify class distribution since exceptionally poor ratings, which make up a large number of releases, are absent. 



In [ ]:
# Visualize class distribution.
class_counts = analysis_df['Review_Category'].value_counts().sort_index()
bars = plt.bar(class_counts.index, class_counts.values, color='skyblue', edgecolor='black')
plt.bar_label(bars)
plt.savefig('output/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# Percent of data with *High* reviews. 
print("Class distribution (%):")
print((analysis_df['Review_Category'].value_counts(normalize=True) * 100).round(2).to_string())

This carries a glaring class imbalance, with almost 70% of data belonging to 1 of 3 classes. 

Moving forward, we'll create and deploy training sets to evaluate balanced data and unbalanced as a benchmark.

In [ ]:
# Separate features and target.
X = analysis_df.drop(columns=["Review_Category"])
y = analysis_df["Review_Category"]

# Split into train and test sets.
split_amount = 0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_amount, random_state=42, stratify=y)

We'll balance our data by resampling each class with SMOTE to have the same number of values. 

It's important to strike a fair balance between undersampling and oversampling. *Low* and *Medium* need greater representation, but we don't want to squash our *High* data.

In [ ]:
# Start by importing SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

# Combine X_train and y_train.
train_df = pd.concat([X_train, y_train], axis=1)

# Split into class subsets.
high_df = train_df[train_df["Review_Category"] == "High"]
med_df  = train_df[train_df["Review_Category"] == "Medium"]
low_df  = train_df[train_df["Review_Category"] == "Low"]

In [ ]:
# Downsample *High* class to 900.
high_resample = resample(high_df, replace=False, n_samples=900, random_state=42)

# Recombine with Medium and Low.
train_balanced = pd.concat([high_resample, med_df, low_df])

# Separate back into X and y.
X_bal = train_balanced.drop(columns=["Review_Category"])
y_bal = train_balanced["Review_Category"]

# Oversample *Low* & *Medium* classes. 
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_bal, y_bal)

In [ ]:
# Verify balanced distribution.
print(y_train_bal.value_counts().to_string())

Classes have been resampled to contain the same number of values (900).

- The *High* class was randomly reduced to 900 samples to prevent it from dominating the model.
- The *Medium* and *Low* classes were oversampled to 900 each using SMOTE.

We're using SMOTE, as it preserves the integrity of the data by generating new, realistic samples based on learned patterns. While imperfect, we need lots of synthetic data, and having a significant amount of duplicate data could have major consequences. 


# Section 3: Creating Our Model

##### **Random Forest Classifier**

- Captures non-linear relationships and handles both continuous and categorical features with minimal preprocessing.
- Reduces overfitting and remains stable even with noisy or synthetic (SMOTE) data. 

We'll test two versions: one with unbalanced data and weighted classes, and another with balanced data.

In [ ]:
# Train Random Forest on unbalanced data, & weighted classes.
rf_unbalanced = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_unbalanced.fit(X_train, y_train)

# Train Random Forest on balanced data.
rf_balanced = RandomForestClassifier(random_state=42)
rf_balanced.fit(X_train_bal, y_train_bal)

# Unbalanced Accuracy & Classification Report.
y_pred = rf_unbalanced.predict(X_test)
print("\nUnbalanced Classification Report:")
print("-----------------------------------------------------")
print(classification_report(y_test, y_pred))
print("-----------------------------------------------------")

# Balanced Accuracy & Classification Report.
y_pred_bal = rf_balanced.predict(X_test)
print("\nBalanced Classification Report:")
print("-----------------------------------------------------")
print(classification_report(y_test, y_pred_bal))
print("-----------------------------------------------------")

From an initial analysis using classification reports: balancing the data noticeably decreased accuracy, but moderately improved *Low* & *Medium* performance. 

For further clarity on how the models differ, we'll create a confusion matrix heatmap for both.

In [ ]:
# Create subplots for side by side comparison.
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Unbalanced model confusion matrix.
cm_unbalanced = confusion_matrix(y_test, y_pred)
sns.heatmap(cm_unbalanced, annot=True, fmt='d', cmap='Blues', xticklabels=['High', 'Low', 'Medium'], yticklabels=['High', 'Low', 'Medium'], ax=axes[0])
axes[0].set_title('Confusion Matrix - Unbalanced Model')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Balanced model confusion matrix.
cm_balanced = confusion_matrix(y_test, y_pred_bal)
sns.heatmap(cm_balanced, annot=True, fmt='d', cmap='Greens', xticklabels=['High', 'Low', 'Medium'], yticklabels=['High', 'Low', 'Medium'], ax=axes[1])
axes[1].set_title('Confusion Matrix - Balanced Model')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.savefig('output/confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

The unbalanced model heavily favors the *High* class, with almost no color in or recognition of *Low* or *Medium* tiles.

The balanced model still greatly favors *High*, but predictions are more evenly distributed, improving minority-class performance at the cost of accuracy. 

##### **Random Forest Interpretation**

Unbalanced Model: (higher overall accuracy, biased to majority)
- Accuracy 0.72, driven by very high recall on *High* (0.96).
- Minority classes suffer: *Low* recall 0.17, *Medium* recall 0.16.
- Macro F1 = 0.44, showing poor average per-class performance.

Balanced Model: (fairer across classes, lower accuracy)
- Accuracy drops to 0.60 as the model no longer over focuses on *High*.
- Minority class recall improves: *Low* 0.40, *Medium* 0.34.
- Macro F1 = 0.46 and macro recall = 0.48, showing more equitable performance.

Trade-offs:
- Unbalanced: higher overall accuracy, weak minority detection, best for maximizing accuracy on the majority class.
- Balanced: stronger class balance, lower accuracy, best if the goal involves the minority classes.

# Section 4: Feature Importance
To further interpret the results, and understand how our model is being built, we'll take a look at what features are actually making an impact.  

In [ ]:
# Feature importances for visualization and comparison.
rf_unbalanced_fi = pd.Series(rf_unbalanced.feature_importances_, index=X_train.columns).sort_values(ascending=False)
rf_balanced_fi = pd.Series(rf_balanced.feature_importances_, index=X_train_bal.columns).sort_values(ascending=False)

In [ ]:
# Visualising Unbalanced Feature Importance.
plt.figure(figsize=(10, 6))
bars = plt.barh(rf_unbalanced_fi.sort_values(ascending=True).index, rf_unbalanced_fi.sort_values(ascending=True).values, color="steelblue")

# Add value annotations to bars.
plt.bar_label(bars, fmt='%.3f', fontsize=9)

plt.title("Feature Importance - Unbalanced Model")
plt.xlabel("Importance")

plt.savefig('output/barh_feature_importance_unbalanced.png', dpi=150)
plt.show()

The unbalanced model heavily relies on popularity (Log Reviews). Log Price, Discount, Month, and Year, are all moderately impactful, but without reviews, predictive power would be minimal. 

In [ ]:
# Visualising Balanced Feature Importance.
plt.figure(figsize=(10, 6))
bars = plt.barh(rf_balanced_fi.sort_values(ascending=True).index, rf_balanced_fi.sort_values(ascending=True).values, color="seagreen")

# Add value annotations to bars.
plt.bar_label(bars, fmt='%.3f', fontsize=9)

plt.title("Feature Importance - Balanced Model")
plt.xlabel("Importance")

plt.savefig('output/barh_feature_importance_balanced.png', dpi=150, bbox_inches='tight')
plt.show()

With the balanced model, reliance on Log Reviews is reduced, and feature importance is distributed a little more evenly. The distribution overall looks much more uniform and usable than unbalanced. 

In [ ]:
# Create feature importance DataFrame.
unbal_series = pd.Series(rf_unbalanced.feature_importances_, index=X_train.columns, name="Unbalanced")
bal_series = pd.Series(rf_balanced.feature_importances_, index=X_train_bal.columns, name="Balanced")
all_feats = unbal_series.index.union(bal_series.index)
fi_df = pd.concat([unbal_series.reindex(all_feats), bal_series.reindex(all_feats)], axis=1)
fi_df["Difference"] = fi_df["Balanced"] - fi_df["Unbalanced"] # Adding difference column. 

In [ ]:
# Print comprehensive dataframe. 
print("---------- Feature Importance Data ---------------\n ")
print(fi_df)
print("--------------------------------------------------")

Feature Importance:
- Log Reviews is the strongest predictor, but its influence drops after balancing.
- Log Price and Discount gain importance post-balancing, suggesting price/discount are more impactful once the class bias is reduced.
- Month & Year are moderately impactful, with small decreases indicating modest seasonality/temporal effects.
- OS features are minimally impactful overall; MacOS slightly ticks up, but remains minor. Windows, Linux, and OS_Support_Count are negligible.

Insights & Takeaways:
- With unbalanced data, the model leans heavily on engagement/popularity (Log Reviews). After balancing, it spreads importance toward pricing and discounting, implying these factors better differentiate Medium/Low vs High when class bias is controlled.
- Many predictors here are pleasantly surprising. While OS support proved nearly redundant, I anticipated predictive power would come almost exclusively from Log Reviews and Discount.

# Section 5: Additional Models

Before concluding, we’ll briefly run a few alternative classifiers. It’s important to verify that Random Forest isn’t the only model producing certain results. The goal isn’t to fully tune or optimize these models, but to ensure our findings aren’t unique to one algorithm, and that we chose the right model. 

##### **Logistic Regression**

In [ ]:
# Logistic Regression.
log_reg = LogisticRegression(max_iter=100000, class_weight='balanced', random_state=42) 
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

print("\nLogReg Classification Report:")
print("-----------------------------------------------------")
print(classification_report(y_test, y_pred_log))
print("-----------------------------------------------------")

- Performs noticeably worse than Random Forest. While it achieves high precision on the *High* class, recall is poor, accuracy is poor, and performance on the minority classes remained weak. 
- Overall accuracy and macro F1 show the model struggled to capture the complexity of the data compared to a Random Forest.
- Raised a convergence warning at 1000 iterations. For a fairer comparison, increased the iteration limit to 100000. While not the central focus, this ensured stable results without a warning. 

##### **K-Nearest Neighbors**

In [ ]:
# KNN Classifier. 
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("\nKNN Classification Report:")
print("-----------------------------------------------------")
print(classification_report(y_test, y_pred_knn))
print("-----------------------------------------------------")

- While it achieves strong recall on the *High* class, precision and recall are very poor on the minority classes.
- Overall accuracy (0.67) looks decent, but the low macro F1 (0.37) shows that it fails to generalize across classes, relying heavily on the dominant class.

# Conclusion

The goal of this project was to analyze whether structured attributes have predictive power when predicting game reviews using Steam sales data. Through machine learning analysis, we've determined that these variables *do* carry, although limited, predictive weight.

**Key Findings:** Random Forest models achieved meaningful predictive performance:
- Unbalanced Model: 72% accuracy with strong recall on majority class (*High* reviews: 96%).
- Balanced Model: 60% accuracy with more equitable performance across all classes.
- Macro F1 scores: 0.44 (unbalanced) vs 0.46 (balanced), showing balanced model's superior class generalization.
- Comparators: Logistic Regression and KNN confirmed the pattern, both leaned on the majority class and underperformed RF on macro F1.

**Feature Interpretability:**
- Log Reviews: (32.2% → 28.1% importance) - Player engagement is the strongest predictor.
- Log Price: (19.4% → 22.0% importance) - Pricing strategy significantly impacts reviews.
- Discount: (15.7% → 17.7% importance) - Sales promotions affect, or reflect, player sentiment.
- Month/Year: (14.3% / 12.8% importance) - Release patterns do matter.

**Human Predictability:** Moderate performance suggests human sentiment has both predictable and unpredictable elements:
- Predictable aspects: Engagement metrics, pricing, and temporal factors show clear patterns.
- Unpredictable aspects: The remaining 28-40% error suggests factors like game quality, genre preferences, marketing effectiveness, and individual taste variations.

**Limitations & Next Steps:**

- Minority classes remain hard; rely on macro/weighted F1 and confusion matrices, not accuracy alone.
- Add more features (genre, developer reputation, text reviews).
- Create visuals for alternative models, and explore them further. 

**Refrences**
- Dataset on Kaggle (updated weekly): https://www.kaggle.com/datasets/benjaminlundkvist/steam-sales-historical-dataset.
- Steam store: https://store.steampowered.com/
- SMOTE: https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
- matplotlib bar_label: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar_label.html
